# Day8_2: 시계열 시각화 + 프로젝트 (Plotly 고급)

## 📚 학습 목표

**Part 1: 기초**
1. px.line() 고급 파라미터 활용하기
2. Range Slider로 시간 범위 선택 구현하기
3. 다중 Y축 차트 만들기
4. Time Series Annotations 추가하기
5. Candlestick 차트 기본 익히기

**Part 2: 심화**
1. 이동평균(MA) 계산 및 시각화하기
2. yfinance로 실시간 금융 데이터 수집하기
3. 볼린저 밴드 구현하기
4. 종합 대시보드 구축하기 (암호화폐 분석 프로젝트)

---

## 🎯 왜 이것을 배우나요?

| 개념 | 실무 활용 | 예시 |
|------|----------|------|
| Range Slider | 대규모 시계열 데이터 탐색 | 1년치 주가 데이터에서 특정 월만 확대 |
| 다중 Y축 | 단위가 다른 지표 동시 표시 | 주가($)와 거래량(개) 같이 보기 |
| Candlestick | 금융 시장 분석 필수 차트 | 주식/암호화폐 시가/종가/고가/저가 표현 |
| Annotations | 중요 이벤트 강조 | 실적 발표일, 뉴스 발생 시점 표시 |
| 이동평균 | 추세 파악 및 매매 신호 | 단기/장기 이평선 교차로 매수/매도 판단 |

**분석가 관점**: 시계열 데이터는 패턴과 이상치를 시각적으로 발견하는 것이 핵심! Plotly의 인터랙티브 기능으로 숨겨진 트렌드를 찾아냅니다.

---

# Part 1: 기초

---

## 1.1 px.line() 고급 파라미터

### 기본 시계열 차트 복습

먼저 간단한 시계열 차트를 만들어봅시다.

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta

# 샘플 주가 데이터 생성
dates = pd.date_range('2024-01-01', periods=100, freq='D')
prices = 100 + (dates.day_of_year / 10) + pd.Series(range(100)).apply(lambda x: x % 10 - 5)

df = pd.DataFrame({
    'date': dates,
    'price': prices
})

# 기본 라인 차트
fig = px.line(df, x='date', y='price', title='기본 시계열 차트')
fig.show()

print(f"데이터 범위: {df['date'].min().date()} ~ {df['date'].max().date()}")
print(f"평균 가격: ${df['price'].mean():.2f}")

### 고급 파라미터: markers, line_shape, color

차트를 더욱 풍부하게 만드는 파라미터들을 알아봅시다.

In [ ]:
# 마커 추가 + 선 스타일 변경
fig = px.line(
    df, 
    x='date', 
    y='price',
    title='고급 스타일 적용',
    markers=True,  # 데이터 포인트 표시
    line_shape='spline'  # 부드러운 곡선
)

# 색상 및 선 두께 커스터마이징
fig.update_traces(
    line_color='#FF6B6B',
    line_width=3,
    marker=dict(size=8, color='#4ECDC4', line=dict(width=2, color='white'))
)

fig.show()

### 💡 실무 예시: 주간 매출 트렌드 강조

주말 매출이 높은 패턴을 색상으로 구분해봅시다.

In [ ]:
# 요일별 매출 데이터
df_sales = pd.DataFrame({
    'date': pd.date_range('2024-01-01', periods=30, freq='D'),
    'sales': [120, 130, 140, 150, 160, 200, 210] * 4 + [120, 130]  # 주말 높음
})
df_sales['is_weekend'] = df_sales['date'].dt.dayofweek >= 5
df_sales['day_type'] = df_sales['is_weekend'].map({True: '주말', False: '평일'})

# 평일/주말 구분 시각화
fig = px.line(
    df_sales,
    x='date',
    y='sales',
    color='day_type',
    title='평일 vs 주말 매출 트렌드',
    labels={'sales': '매출 (만원)', 'date': '날짜', 'day_type': '구분'},
    markers=True
)

fig.show()

print(f"평일 평균: {df_sales[~df_sales['is_weekend']]['sales'].mean():.1f}만원")
print(f"주말 평균: {df_sales[df_sales['is_weekend']]['sales'].mean():.1f}만원")

---

## 1.2 Range Slider 추가

### Range Slider란?

차트 하단에 슬라이더를 추가하여 **사용자가 시간 범위를 직접 선택**할 수 있게 합니다.
대규모 시계열 데이터 탐색에 필수!

In [ ]:
# 1년치 데이터 생성
dates_long = pd.date_range('2023-01-01', periods=365, freq='D')
prices_long = 100 + (dates_long.day_of_year / 5) + pd.Series(range(365)).apply(lambda x: (x % 30 - 15))

df_long = pd.DataFrame({
    'date': dates_long,
    'price': prices_long
})

# Range Slider 추가
fig = px.line(df_long, x='date', y='price', title='1년 주가 데이터 (Range Slider 활용)')

fig.update_xaxes(
    rangeslider_visible=True,  # Range Slider 활성화
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1개월", step="month", stepmode="backward"),
            dict(count=3, label="3개월", step="month", stepmode="backward"),
            dict(count=6, label="6개월", step="month", stepmode="backward"),
            dict(step="all", label="전체")
        ])
    )
)

fig.show()

### 💡 실무 예시: 분기별 매출 분석

버튼 클릭으로 분기별 매출을 빠르게 비교합니다.

In [ ]:
# 분기별 매출 데이터
df_quarterly = pd.DataFrame({
    'date': pd.date_range('2023-01-01', periods=365, freq='D'),
    'revenue': 1000 + (pd.date_range('2023-01-01', periods=365, freq='D').quarter * 200) + 
               pd.Series(range(365)).apply(lambda x: (x % 90 - 45))
})

fig = px.line(
    df_quarterly,
    x='date',
    y='revenue',
    title='2023년 분기별 매출 추이',
    labels={'revenue': '매출 (만원)', 'date': '날짜'}
)

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1분기", step="month", stepmode="todate"),
            dict(count=3, label="Q1", step="month", stepmode="backward"),
            dict(count=6, label="상반기", step="month", stepmode="backward"),
            dict(step="all", label="전체")
        ])
    )
)

fig.show()

---

## 1.3 다중 Y축 (Multiple Y-axes)

### 왜 필요한가?

**단위가 다른 두 지표**를 하나의 차트에 표시할 때 사용합니다.
예: 주가($)와 거래량(천 개), 매출(백만)과 이익률(%)

In [ ]:
# 주가 + 거래량 데이터
df_stock = pd.DataFrame({
    'date': pd.date_range('2024-01-01', periods=60, freq='D'),
    'price': 150 + pd.Series(range(60)).apply(lambda x: (x % 20 - 10)),
    'volume': 10000 + pd.Series(range(60)).apply(lambda x: (x % 15 - 7.5) * 1000)
})

# Figure 생성 (graph_objects 사용)
fig = go.Figure()

# 주가 (왼쪽 Y축)
fig.add_trace(go.Scatter(
    x=df_stock['date'],
    y=df_stock['price'],
    name='주가',
    line=dict(color='#4ECDC4', width=3),
    yaxis='y1'
))

# 거래량 (오른쪽 Y축)
fig.add_trace(go.Scatter(
    x=df_stock['date'],
    y=df_stock['volume'],
    name='거래량',
    line=dict(color='#FF6B6B', width=2, dash='dot'),
    yaxis='y2'
))

# 레이아웃 설정
fig.update_layout(
    title='주가 및 거래량 추이 (다중 Y축)',
    xaxis=dict(title='날짜'),
    yaxis=dict(title='주가 ($)', side='left', showgrid=False),
    yaxis2=dict(title='거래량 (개)', side='right', overlaying='y', showgrid=False),
    hovermode='x unified'
)

fig.show()

### 💡 실무 예시: 매출과 수익률 동시 분석

In [ ]:
# 월별 매출 및 수익률 데이터
df_profit = pd.DataFrame({
    'month': pd.date_range('2023-01-01', periods=12, freq='ME'),
    'revenue': [5000, 5200, 5800, 6100, 6500, 7000, 7200, 7500, 7800, 8000, 8500, 9000],
    'profit_margin': [12.5, 13.0, 14.2, 14.8, 15.1, 15.5, 14.9, 15.8, 16.2, 16.0, 16.5, 17.0]
})

fig = go.Figure()

# 매출 (막대 그래프, 왼쪽 Y축)
fig.add_trace(go.Bar(
    x=df_profit['month'],
    y=df_profit['revenue'],
    name='매출',
    marker_color='#95E1D3',
    yaxis='y1'
))

# 수익률 (선 그래프, 오른쪽 Y축)
fig.add_trace(go.Scatter(
    x=df_profit['month'],
    y=df_profit['profit_margin'],
    name='수익률',
    line=dict(color='#F38181', width=3),
    mode='lines+markers',
    marker=dict(size=10),
    yaxis='y2'
))

fig.update_layout(
    title='월별 매출 및 수익률 분석',
    xaxis=dict(title='월'),
    yaxis=dict(title='매출 (만원)', side='left'),
    yaxis2=dict(title='수익률 (%)', side='right', overlaying='y'),
    hovermode='x unified',
    height=500
)

fig.show()

---

## 1.4 Time Series Annotations

### Annotation이란?

차트에 **텍스트, 화살표, 도형**을 추가하여 **중요한 이벤트를 강조**합니다.
예: 신제품 출시일, 실적 발표일, 뉴스 이벤트

In [ ]:
# 주가 데이터 + 이벤트
df_events = pd.DataFrame({
    'date': pd.date_range('2024-01-01', periods=90, freq='D'),
    'price': 100 + pd.Series(range(90)).apply(lambda x: (x / 5) + (x % 20 - 10))
})

# 기본 차트
fig = px.line(df_events, x='date', y='price', title='주가 차트 with Annotations')

# Annotation 추가
fig.add_annotation(
    x='2024-01-15',
    y=df_events[df_events['date'] == '2024-01-15']['price'].values[0],
    text="📢 신제품 출시",
    showarrow=True,
    arrowhead=2,
    arrowcolor='#FF6B6B',
    ax=0,
    ay=-40,
    font=dict(size=12, color='#FF6B6B')
)

fig.add_annotation(
    x='2024-02-20',
    y=df_events[df_events['date'] == '2024-02-20']['price'].values[0],
    text="💰 실적 발표 (상승)",
    showarrow=True,
    arrowhead=2,
    arrowcolor='#4ECDC4',
    ax=0,
    ay=-40,
    font=dict(size=12, color='#4ECDC4')
)

fig.show()

### 수직선으로 이벤트 표시

In [ ]:
# 동일한 데이터
fig = px.line(df_events, x='date', y='price', title='이벤트 타임라인 (수직선 사용)')

# 수직선 추가 (vline)
fig.add_vline(
    x=pd.Timestamp('2024-01-15').timestamp() * 1000,  # milliseconds
    line_dash="dash",
    line_color="red",
    annotation_text="신제품 출시",
    annotation_position="top left"
)

fig.add_vline(
    x=pd.Timestamp('2024-02-20').timestamp() * 1000,
    line_dash="dash",
    line_color="green",
    annotation_text="실적 발표",
    annotation_position="top right"
)

fig.show()

### 💡 실무 예시: 마케팅 캠페인 효과 분석

In [ ]:
# 웹사이트 방문자 수 데이터
df_visitors = pd.DataFrame({
    'date': pd.date_range('2024-01-01', periods=60, freq='D'),
    'visitors': 5000 + pd.Series(range(60)).apply(lambda x: (x * 50) + (x % 10 - 5) * 200)
})

# 캠페인 이벤트
campaigns = [
    {'date': '2024-01-10', 'name': '🎉 신년 할인 이벤트'},
    {'date': '2024-01-25', 'name': '📧 이메일 마케팅'},
    {'date': '2024-02-14', 'name': '💝 발렌타인 프로모션'}
]

fig = px.line(
    df_visitors,
    x='date',
    y='visitors',
    title='마케팅 캠페인별 웹사이트 방문자 추이',
    labels={'visitors': '방문자 수', 'date': '날짜'}
)

# 각 캠페인에 수직선 추가
for campaign in campaigns:
    fig.add_vline(
        x=pd.Timestamp(campaign['date']).timestamp() * 1000,
        line_dash="dot",
        line_color="orange",
        annotation_text=campaign['name'],
        annotation_position="top"
    )

fig.show()

---

## 1.5 Candlestick 차트 기본

### Candlestick이란?

주식/암호화폐 시장에서 가장 많이 쓰는 차트!
**시가, 고가, 저가, 종가 (OHLC)**를 하나의 캔들로 표현합니다.

- **초록 캔들**: 종가 > 시가 (상승)
- **빨강 캔들**: 종가 < 시가 (하락)

In [ ]:
# OHLC 데이터 생성
import numpy as np

dates_ohlc = pd.date_range('2024-01-01', periods=30, freq='D')
open_prices = 100 + np.random.randn(30).cumsum()
close_prices = open_prices + np.random.randn(30) * 2
high_prices = np.maximum(open_prices, close_prices) + np.abs(np.random.randn(30))
low_prices = np.minimum(open_prices, close_prices) - np.abs(np.random.randn(30))

df_ohlc = pd.DataFrame({
    'date': dates_ohlc,
    'open': open_prices,
    'high': high_prices,
    'low': low_prices,
    'close': close_prices
})

# Candlestick 차트
fig = go.Figure(data=[go.Candlestick(
    x=df_ohlc['date'],
    open=df_ohlc['open'],
    high=df_ohlc['high'],
    low=df_ohlc['low'],
    close=df_ohlc['close'],
    name='OHLC'
)])

fig.update_layout(
    title='Candlestick 차트 예시',
    xaxis_title='날짜',
    yaxis_title='가격 ($)',
    xaxis_rangeslider_visible=False,  # 하단 슬라이더 숨김
    height=500
)

fig.show()

print(f"최고가: ${df_ohlc['high'].max():.2f}")
print(f"최저가: ${df_ohlc['low'].min():.2f}")

### 💡 실무 예시: 비트코인 주간 차트

In [ ]:
# 비트코인 가상 데이터 (주간)
btc_dates = pd.date_range('2024-01-01', periods=20, freq='W')
btc_open = 40000 + np.random.randn(20).cumsum() * 500
btc_close = btc_open + np.random.randn(20) * 1000
btc_high = np.maximum(btc_open, btc_close) + np.abs(np.random.randn(20)) * 500
btc_low = np.minimum(btc_open, btc_close) - np.abs(np.random.randn(20)) * 500

df_btc = pd.DataFrame({
    'date': btc_dates,
    'open': btc_open,
    'high': btc_high,
    'low': btc_low,
    'close': btc_close
})

fig = go.Figure(data=[go.Candlestick(
    x=df_btc['date'],
    open=df_btc['open'],
    high=df_btc['high'],
    low=df_btc['low'],
    close=df_btc['close'],
    increasing_line_color='#26A69A',  # 상승 색상
    decreasing_line_color='#EF5350',  # 하락 색상
    name='BTC'
)])

fig.update_layout(
    title='비트코인 주간 차트 (Candlestick)',
    xaxis_title='주',
    yaxis_title='가격 ($)',
    xaxis_rangeslider_visible=True,
    height=600
)

fig.show()

# 통계
print(f"기간: {df_btc['date'].min().date()} ~ {df_btc['date'].max().date()}")
print(f"최고가: ${df_btc['high'].max():,.0f}")
print(f"최저가: ${df_btc['low'].min():,.0f}")
print(f"변동성: {((df_btc['high'].max() - df_btc['low'].min()) / df_btc['low'].min() * 100):.2f}%")

---

# Part 2: 심화

---

## 2.1 이동평균(MA) 계산 및 시각화

### 이동평균이란?

**일정 기간의 평균**을 계산하여 **추세를 파악**하는 기술적 지표입니다.

- **단기 이평선** (5일, 20일): 빠른 반응
- **장기 이평선** (60일, 120일): 추세 확인
- **골든 크로스**: 단기 > 장기 교차 (매수 신호)
- **데드 크로스**: 단기 < 장기 교차 (매도 신호)

In [ ]:
# 주가 데이터 생성
dates_ma = pd.date_range('2023-01-01', periods=200, freq='D')
prices_ma = 100 + np.random.randn(200).cumsum() * 2

df_ma = pd.DataFrame({
    'date': dates_ma,
    'price': prices_ma
})

# 이동평균 계산
df_ma['MA5'] = df_ma['price'].rolling(window=5).mean()    # 5일 이평선
df_ma['MA20'] = df_ma['price'].rolling(window=20).mean()  # 20일 이평선
df_ma['MA60'] = df_ma['price'].rolling(window=60).mean()  # 60일 이평선

print(df_ma.head(10))

In [ ]:
# 이동평균선 시각화
fig = go.Figure()

# 주가
fig.add_trace(go.Scatter(
    x=df_ma['date'],
    y=df_ma['price'],
    name='주가',
    line=dict(color='lightgray', width=1),
    opacity=0.5
))

# MA5 (단기)
fig.add_trace(go.Scatter(
    x=df_ma['date'],
    y=df_ma['MA5'],
    name='MA5 (단기)',
    line=dict(color='#FF6B6B', width=2)
))

# MA20 (중기)
fig.add_trace(go.Scatter(
    x=df_ma['date'],
    y=df_ma['MA20'],
    name='MA20 (중기)',
    line=dict(color='#4ECDC4', width=2)
))

# MA60 (장기)
fig.add_trace(go.Scatter(
    x=df_ma['date'],
    y=df_ma['MA60'],
    name='MA60 (장기)',
    line=dict(color='#95E1D3', width=2)
))

fig.update_layout(
    title='이동평균선 차트',
    xaxis_title='날짜',
    yaxis_title='가격 ($)',
    hovermode='x unified',
    height=500
)

fig.show()

### 💡 실무 예시: 골든 크로스 / 데드 크로스 탐지

In [ ]:
# 골든/데드 크로스 탐지
df_ma['signal'] = 0
df_ma.loc[df_ma['MA5'] > df_ma['MA20'], 'signal'] = 1   # 골든 크로스
df_ma.loc[df_ma['MA5'] < df_ma['MA20'], 'signal'] = -1  # 데드 크로스

# 크로스 발생 지점 찾기 (신호 변화)
df_ma['signal_change'] = df_ma['signal'].diff()
golden_cross = df_ma[df_ma['signal_change'] == 2]  # -1 → 1
dead_cross = df_ma[df_ma['signal_change'] == -2]   # 1 → -1

print(f"골든 크로스 발생: {len(golden_cross)}회")
print(f"데드 크로스 발생: {len(dead_cross)}회")

# 시각화
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_ma['date'], y=df_ma['price'],
    name='주가', line=dict(color='lightgray')
))

fig.add_trace(go.Scatter(
    x=df_ma['date'], y=df_ma['MA5'],
    name='MA5', line=dict(color='#FF6B6B')
))

fig.add_trace(go.Scatter(
    x=df_ma['date'], y=df_ma['MA20'],
    name='MA20', line=dict(color='#4ECDC4')
))

# 골든 크로스 표시
fig.add_trace(go.Scatter(
    x=golden_cross['date'],
    y=golden_cross['price'],
    mode='markers',
    name='골든 크로스',
    marker=dict(symbol='triangle-up', size=15, color='green')
))

# 데드 크로스 표시
fig.add_trace(go.Scatter(
    x=dead_cross['date'],
    y=dead_cross['price'],
    mode='markers',
    name='데드 크로스',
    marker=dict(symbol='triangle-down', size=15, color='red')
))

fig.update_layout(
    title='골든 크로스 & 데드 크로스 탐지',
    xaxis_title='날짜',
    yaxis_title='가격 ($)',
    hovermode='x unified',
    height=500
)

fig.show()

---

## 2.2 yfinance로 실시간 금융 데이터 수집

### yfinance란?

**Yahoo Finance API**를 통해 **무료로** 주식/암호화폐 데이터를 다운로드하는 라이브러리입니다.

설치: `pip install yfinance`

In [ ]:
# yfinance 설치 (필요 시)
# !pip install yfinance

import yfinance as yf

# 애플 주식 데이터 다운로드 (최근 3개월)
aapl = yf.Ticker("AAPL")
df_aapl = aapl.history(period="3mo")

print(df_aapl.head())
print(f"\n데이터 크기: {df_aapl.shape}")
print(f"컬럼: {df_aapl.columns.tolist()}")

In [ ]:
# 애플 주가 Candlestick 차트
fig = go.Figure(data=[go.Candlestick(
    x=df_aapl.index,
    open=df_aapl['Open'],
    high=df_aapl['High'],
    low=df_aapl['Low'],
    close=df_aapl['Close'],
    name='AAPL'
)])

fig.update_layout(
    title='애플(AAPL) 최근 3개월 주가',
    xaxis_title='날짜',
    yaxis_title='가격 ($)',
    xaxis_rangeslider_visible=True,
    height=600
)

fig.show()

### 💡 실무 예시: 비트코인 실시간 데이터 + 거래량

In [ ]:
# 비트코인 USD 데이터 (최근 1개월)
btc = yf.Ticker("BTC-USD")
df_btc_real = btc.history(period="1mo")

# 이동평균 계산
df_btc_real['MA7'] = df_btc_real['Close'].rolling(window=7).mean()
df_btc_real['MA21'] = df_btc_real['Close'].rolling(window=21).mean()

# Candlestick + 이동평균
fig = go.Figure()

# Candlestick
fig.add_trace(go.Candlestick(
    x=df_btc_real.index,
    open=df_btc_real['Open'],
    high=df_btc_real['High'],
    low=df_btc_real['Low'],
    close=df_btc_real['Close'],
    name='BTC',
    yaxis='y1'
))

# MA7
fig.add_trace(go.Scatter(
    x=df_btc_real.index,
    y=df_btc_real['MA7'],
    name='MA7',
    line=dict(color='orange', width=2),
    yaxis='y1'
))

# MA21
fig.add_trace(go.Scatter(
    x=df_btc_real.index,
    y=df_btc_real['MA21'],
    name='MA21',
    line=dict(color='blue', width=2),
    yaxis='y1'
))

# 거래량 (막대 그래프, 별도 Y축)
fig.add_trace(go.Bar(
    x=df_btc_real.index,
    y=df_btc_real['Volume'],
    name='거래량',
    marker_color='rgba(100,100,100,0.3)',
    yaxis='y2'
))

fig.update_layout(
    title='비트코인(BTC-USD) 최근 1개월 차트',
    xaxis=dict(title='날짜'),
    yaxis=dict(title='가격 ($)', side='left'),
    yaxis2=dict(title='거래량', side='right', overlaying='y', showgrid=False),
    xaxis_rangeslider_visible=False,
    hovermode='x unified',
    height=600
)

fig.show()

print(f"최고가: ${df_btc_real['High'].max():,.2f}")
print(f"최저가: ${df_btc_real['Low'].min():,.2f}")
print(f"현재가: ${df_btc_real['Close'].iloc[-1]:,.2f}")

---

## 2.3 볼린저 밴드 (Bollinger Bands)

### 볼린저 밴드란?

**이동평균 ± (표준편차 × 2)**로 계산되는 변동성 지표입니다.

- **상단 밴드**: MA + 2σ (고평가 영역)
- **중간 밴드**: MA (평균)
- **하단 밴드**: MA - 2σ (저평가 영역)

**해석**:
- 가격이 상단 밴드 도달 → 과매수 (매도 고려)
- 가격이 하단 밴드 도달 → 과매도 (매수 고려)
- 밴드 폭 좁아짐 → 변동성 감소 → 큰 움직임 예고

In [ ]:
# 볼린저 밴드 계산 함수
def calculate_bollinger_bands(df, column='Close', window=20, num_std=2):
    """
    볼린저 밴드 계산
    
    Parameters:
    - df: DataFrame
    - column: 가격 컬럼명
    - window: 이동평균 기간 (기본 20일)
    - num_std: 표준편차 배수 (기본 2)
    """
    df = df.copy()
    df['BB_Middle'] = df[column].rolling(window=window).mean()
    df['BB_Std'] = df[column].rolling(window=window).std()
    df['BB_Upper'] = df['BB_Middle'] + (df['BB_Std'] * num_std)
    df['BB_Lower'] = df['BB_Middle'] - (df['BB_Std'] * num_std)
    return df

# 테스트용 주가 데이터
dates_bb = pd.date_range('2023-01-01', periods=120, freq='D')
prices_bb = 100 + np.random.randn(120).cumsum() * 3

df_bb = pd.DataFrame({
    'date': dates_bb,
    'Close': prices_bb
})

# 볼린저 밴드 계산
df_bb = calculate_bollinger_bands(df_bb, window=20, num_std=2)

print(df_bb[['date', 'Close', 'BB_Upper', 'BB_Middle', 'BB_Lower']].tail(10))

In [ ]:
# 볼린저 밴드 시각화
fig = go.Figure()

# 상단 밴드
fig.add_trace(go.Scatter(
    x=df_bb['date'],
    y=df_bb['BB_Upper'],
    name='상단 밴드',
    line=dict(color='red', width=1, dash='dash'),
    opacity=0.5
))

# 중간 밴드 (이동평균)
fig.add_trace(go.Scatter(
    x=df_bb['date'],
    y=df_bb['BB_Middle'],
    name='중간 밴드 (MA20)',
    line=dict(color='blue', width=2)
))

# 하단 밴드
fig.add_trace(go.Scatter(
    x=df_bb['date'],
    y=df_bb['BB_Lower'],
    name='하단 밴드',
    line=dict(color='green', width=1, dash='dash'),
    fill='tonexty',  # 상단 밴드와의 사이 채우기
    fillcolor='rgba(100,100,100,0.1)',
    opacity=0.5
))

# 주가
fig.add_trace(go.Scatter(
    x=df_bb['date'],
    y=df_bb['Close'],
    name='주가',
    line=dict(color='black', width=2)
))

fig.update_layout(
    title='볼린저 밴드 (Bollinger Bands)',
    xaxis_title='날짜',
    yaxis_title='가격 ($)',
    hovermode='x unified',
    height=500
)

fig.show()

### 💡 실무 예시: 볼린저 밴드 매매 신호 탐지

In [ ]:
# 과매수/과매도 탐지
df_bb['signal'] = 'Hold'
df_bb.loc[df_bb['Close'] > df_bb['BB_Upper'], 'signal'] = 'Overbought'  # 과매수
df_bb.loc[df_bb['Close'] < df_bb['BB_Lower'], 'signal'] = 'Oversold'   # 과매도

overbought = df_bb[df_bb['signal'] == 'Overbought']
oversold = df_bb[df_bb['signal'] == 'Oversold']

print(f"과매수 신호: {len(overbought)}회")
print(f"과매도 신호: {len(oversold)}회")

# 시각화 with 신호 표시
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_bb['date'], y=df_bb['BB_Upper'],
    name='상단', line=dict(color='red', dash='dash'), opacity=0.5
))

fig.add_trace(go.Scatter(
    x=df_bb['date'], y=df_bb['BB_Middle'],
    name='MA20', line=dict(color='blue')
))

fig.add_trace(go.Scatter(
    x=df_bb['date'], y=df_bb['BB_Lower'],
    name='하단', line=dict(color='green', dash='dash'),
    fill='tonexty', fillcolor='rgba(100,100,100,0.1)', opacity=0.5
))

fig.add_trace(go.Scatter(
    x=df_bb['date'], y=df_bb['Close'],
    name='주가', line=dict(color='black')
))

# 과매수 신호
fig.add_trace(go.Scatter(
    x=overbought['date'],
    y=overbought['Close'],
    mode='markers',
    name='과매수 (매도 고려)',
    marker=dict(symbol='triangle-down', size=12, color='red')
))

# 과매도 신호
fig.add_trace(go.Scatter(
    x=oversold['date'],
    y=oversold['Close'],
    mode='markers',
    name='과매도 (매수 고려)',
    marker=dict(symbol='triangle-up', size=12, color='green')
))

fig.update_layout(
    title='볼린저 밴드 매매 신호',
    xaxis_title='날짜',
    yaxis_title='가격 ($)',
    hovermode='x unified',
    height=500
)

fig.show()

---

## 2.4 종합 프로젝트: 암호화폐 가격 분석 대시보드

### 프로젝트 목표

**yfinance로 암호화폐 데이터를 수집**하고, **Plotly로 인터랙티브 대시보드**를 만들어봅시다!

**구현 요소**:
1. yfinance로 BTC, ETH 데이터 수집
2. Candlestick 차트 + 거래량
3. 이동평균선 (MA7, MA21)
4. 볼린저 밴드
5. 매매 신호 표시 (골든/데드 크로스, 과매수/과매도)
6. 주요 통계 출력

In [ ]:
# Step 1: 데이터 수집
print("📡 데이터 수집 중...")

# 비트코인
btc = yf.Ticker("BTC-USD")
df_crypto = btc.history(period="3mo")
df_crypto = df_crypto.reset_index()

print(f"✅ 데이터 수집 완료: {len(df_crypto)}일치")
print(f"기간: {df_crypto['Date'].min().date()} ~ {df_crypto['Date'].max().date()}")
print(df_crypto.head())

In [ ]:
# Step 2: 기술적 지표 계산
print("📊 기술적 지표 계산 중...")

# 이동평균
df_crypto['MA7'] = df_crypto['Close'].rolling(window=7).mean()
df_crypto['MA21'] = df_crypto['Close'].rolling(window=21).mean()

# 볼린저 밴드
df_crypto = calculate_bollinger_bands(df_crypto, window=20, num_std=2)

# 골든/데드 크로스
df_crypto['signal_ma'] = 0
df_crypto.loc[df_crypto['MA7'] > df_crypto['MA21'], 'signal_ma'] = 1
df_crypto.loc[df_crypto['MA7'] < df_crypto['MA21'], 'signal_ma'] = -1
df_crypto['signal_ma_change'] = df_crypto['signal_ma'].diff()

golden_cross_crypto = df_crypto[df_crypto['signal_ma_change'] == 2]
dead_cross_crypto = df_crypto[df_crypto['signal_ma_change'] == -2]

# 볼린저 밴드 신호
df_crypto['signal_bb'] = 'Hold'
df_crypto.loc[df_crypto['Close'] > df_crypto['BB_Upper'], 'signal_bb'] = 'Overbought'
df_crypto.loc[df_crypto['Close'] < df_crypto['BB_Lower'], 'signal_bb'] = 'Oversold'

overbought_crypto = df_crypto[df_crypto['signal_bb'] == 'Overbought']
oversold_crypto = df_crypto[df_crypto['signal_bb'] == 'Oversold']

print(f"✅ 지표 계산 완료")
print(f"골든 크로스: {len(golden_cross_crypto)}회")
print(f"데드 크로스: {len(dead_cross_crypto)}회")
print(f"과매수: {len(overbought_crypto)}회")
print(f"과매도: {len(oversold_crypto)}회")

In [ ]:
# Step 3: 종합 대시보드 생성
from plotly.subplots import make_subplots

# 2행 구성 (상단: 가격, 하단: 거래량)
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    row_heights=[0.7, 0.3],
    subplot_titles=('비트코인 가격 차트', '거래량')
)

# === 상단: 가격 차트 ===

# Candlestick
fig.add_trace(go.Candlestick(
    x=df_crypto['Date'],
    open=df_crypto['Open'],
    high=df_crypto['High'],
    low=df_crypto['Low'],
    close=df_crypto['Close'],
    name='BTC',
    increasing_line_color='#26A69A',
    decreasing_line_color='#EF5350'
), row=1, col=1)

# MA7
fig.add_trace(go.Scatter(
    x=df_crypto['Date'], y=df_crypto['MA7'],
    name='MA7', line=dict(color='orange', width=2)
), row=1, col=1)

# MA21
fig.add_trace(go.Scatter(
    x=df_crypto['Date'], y=df_crypto['MA21'],
    name='MA21', line=dict(color='blue', width=2)
), row=1, col=1)

# 볼린저 밴드
fig.add_trace(go.Scatter(
    x=df_crypto['Date'], y=df_crypto['BB_Upper'],
    name='BB 상단', line=dict(color='red', dash='dot', width=1), opacity=0.5
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=df_crypto['Date'], y=df_crypto['BB_Lower'],
    name='BB 하단', line=dict(color='green', dash='dot', width=1),
    fill='tonexty', fillcolor='rgba(150,150,150,0.1)', opacity=0.5
), row=1, col=1)

# 골든 크로스
if len(golden_cross_crypto) > 0:
    fig.add_trace(go.Scatter(
        x=golden_cross_crypto['Date'],
        y=golden_cross_crypto['Close'],
        mode='markers',
        name='골든 크로스',
        marker=dict(symbol='triangle-up', size=15, color='green', line=dict(width=2, color='white'))
    ), row=1, col=1)

# 데드 크로스
if len(dead_cross_crypto) > 0:
    fig.add_trace(go.Scatter(
        x=dead_cross_crypto['Date'],
        y=dead_cross_crypto['Close'],
        mode='markers',
        name='데드 크로스',
        marker=dict(symbol='triangle-down', size=15, color='red', line=dict(width=2, color='white'))
    ), row=1, col=1)

# === 하단: 거래량 ===
colors = ['green' if close > open_ else 'red' 
          for close, open_ in zip(df_crypto['Close'], df_crypto['Open'])]

fig.add_trace(go.Bar(
    x=df_crypto['Date'],
    y=df_crypto['Volume'],
    name='거래량',
    marker_color=colors,
    opacity=0.5
), row=2, col=1)

# 레이아웃
fig.update_layout(
    title='🪙 비트코인(BTC-USD) 종합 분석 대시보드',
    xaxis2_title='날짜',
    yaxis_title='가격 ($)',
    yaxis2_title='거래량',
    hovermode='x unified',
    height=800,
    showlegend=True,
    xaxis_rangeslider_visible=False
)

fig.show()

In [ ]:
# Step 4: 주요 통계 리포트
print("\n" + "="*60)
print("📊 비트코인 분석 리포트")
print("="*60)

print(f"\n📅 분석 기간: {df_crypto['Date'].min().date()} ~ {df_crypto['Date'].max().date()}")
print(f"📈 분석 일수: {len(df_crypto)}일")

print(f"\n💰 가격 통계:")
print(f"  - 최고가: ${df_crypto['High'].max():,.2f}")
print(f"  - 최저가: ${df_crypto['Low'].min():,.2f}")
print(f"  - 현재가: ${df_crypto['Close'].iloc[-1]:,.2f}")
print(f"  - 평균가: ${df_crypto['Close'].mean():,.2f}")
print(f"  - 변동성: {((df_crypto['High'].max() - df_crypto['Low'].min()) / df_crypto['Low'].min() * 100):.2f}%")

print(f"\n📊 거래량 통계:")
print(f"  - 최대 거래량: {df_crypto['Volume'].max():,.0f}")
print(f"  - 평균 거래량: {df_crypto['Volume'].mean():,.0f}")

print(f"\n🎯 매매 신호:")
print(f"  - 골든 크로스: {len(golden_cross_crypto)}회")
print(f"  - 데드 크로스: {len(dead_cross_crypto)}회")
print(f"  - 과매수 신호: {len(overbought_crypto)}회")
print(f"  - 과매도 신호: {len(oversold_crypto)}회")

# 최근 신호
if len(df_crypto) > 0:
    latest_signal_ma = "상승 추세" if df_crypto['signal_ma'].iloc[-1] == 1 else "하락 추세" if df_crypto['signal_ma'].iloc[-1] == -1 else "중립"
    latest_signal_bb = df_crypto['signal_bb'].iloc[-1]
    
    print(f"\n⚡ 현재 상태:")
    print(f"  - MA 추세: {latest_signal_ma}")
    print(f"  - BB 신호: {latest_signal_bb}")
    
    if latest_signal_bb == 'Overbought':
        print(f"  - 💡 분석: 과매수 구간, 매도 타이밍 검토 권장")
    elif latest_signal_bb == 'Oversold':
        print(f"  - 💡 분석: 과매도 구간, 매수 타이밍 검토 권장")
    else:
        print(f"  - 💡 분석: 정상 범위 내 거래")

print("\n" + "="*60)

### 💡 프로젝트 확장 아이디어

1. **다중 암호화폐 비교**: BTC, ETH, ADA 동시 분석
2. **자동 알림**: 골든 크로스 발생 시 이메일/슬랙 알림
3. **백테스팅**: 과거 매매 신호로 수익률 시뮬레이션
4. **RSI 지표 추가**: 과매수/과매도 판단 강화
5. **Streamlit 대시보드**: 실시간 업데이트 웹 앱 구축

---

## 🎯 실습 퀴즈

**난이도**: ⭐ (쉬움) ~ ⭐⭐⭐⭐⭐ (어려움)

---

### Q1. Range Slider 추가하기 ⭐

**문제**: 다음 매출 데이터에 Range Slider를 추가하고, "1개월", "3개월", "전체" 버튼을 만드세요.

```python
df_q1 = pd.DataFrame({
    'date': pd.date_range('2024-01-01', periods=180, freq='D'),
    'revenue': 1000 + pd.Series(range(180)).apply(lambda x: x * 5 + (x % 30 - 15) * 10)
})
```

In [ ]:
# 여기에 코드를 작성하세요


### Q2. Annotation 추가하기 ⭐⭐

**문제**: 주가 데이터에서 최고가와 최저가 지점에 Annotation을 추가하세요.

```python
df_q2 = pd.DataFrame({
    'date': pd.date_range('2024-01-01', periods=60, freq='D'),
    'price': [100, 102, 105, 103, 108, 110, 107, 112, 115, 113] * 6
})
```

**힌트**: `df['price'].idxmax()`, `df['price'].idxmin()` 활용

In [ ]:
# 여기에 코드를 작성하세요


### Q3. 다중 Y축 차트 만들기 ⭐⭐

**문제**: 매출(만원)과 이익률(%)을 다중 Y축으로 표시하세요.

```python
df_q3 = pd.DataFrame({
    'month': pd.date_range('2024-01-01', periods=12, freq='ME'),
    'revenue': [5000, 5200, 5800, 6100, 6500, 7000, 7200, 7500, 7800, 8000, 8500, 9000],
    'profit_rate': [10, 12, 14, 13, 15, 16, 14, 17, 18, 16, 19, 20]
})
```

**요구사항**: 매출은 막대 그래프(왼쪽), 이익률은 선 그래프(오른쪽)

In [ ]:
# 여기에 코드를 작성하세요


### Q4. 이동평균 계산하기 ⭐⭐⭐

**문제**: 다음 주가 데이터에서 5일, 20일, 60일 이동평균을 계산하고 한 차트에 표시하세요.

```python
df_q4 = pd.DataFrame({
    'date': pd.date_range('2024-01-01', periods=100, freq='D'),
    'price': 100 + np.random.randn(100).cumsum() * 2
})
```

**기대 결과**: 주가(회색) + MA5(빨강) + MA20(파랑) + MA60(초록)

In [ ]:
# 여기에 코드를 작성하세요


### Q5. Candlestick 차트 생성 ⭐⭐⭐

**문제**: 다음 OHLC 데이터로 Candlestick 차트를 만드세요.

```python
df_q5 = pd.DataFrame({
    'date': pd.date_range('2024-01-01', periods=30, freq='D'),
    'open': [100, 102, 101, 105, 107] * 6,
    'high': [105, 107, 106, 110, 112] * 6,
    'low': [98, 100, 99, 103, 105] * 6,
    'close': [103, 101, 104, 108, 106] * 6
})
```

In [ ]:
# 여기에 코드를 작성하세요


### Q6. 골든/데드 크로스 탐지 ⭐⭐⭐⭐

**문제**: Q4의 데이터에서 MA5와 MA20의 골든 크로스와 데드 크로스를 탐지하고, 차트에 마커로 표시하세요.

**골든 크로스**: MA5가 MA20을 상향 돌파  
**데드 크로스**: MA5가 MA20을 하향 돌파

**기대 결과**: 주가 + 이평선 + 골든 크로스(초록 삼각형 위) + 데드 크로스(빨강 삼각형 아래)

In [ ]:
# 여기에 코드를 작성하세요


### Q7. 볼린저 밴드 계산 ⭐⭐⭐⭐

**문제**: Q4의 데이터에 볼린저 밴드(20일, 2σ)를 계산하고 시각화하세요.

**요구사항**:
- 상단/하단 밴드는 점선
- 밴드 사이는 회색 반투명으로 채우기
- 주가는 검은색 실선

In [ ]:
# 여기에 코드를 작성하세요


### Q8. yfinance 데이터 수집 ⭐⭐⭐⭐

**문제**: yfinance로 **테슬라(TSLA)** 주식의 **최근 6개월** 데이터를 수집하고, Candlestick 차트를 만드세요.

**요구사항**:
1. `period="6mo"` 사용
2. Range Slider 포함
3. 차트 제목에 회사명 표시

In [ ]:
# 여기에 코드를 작성하세요


### Q9. 다중 지표 통합 차트 ⭐⭐⭐⭐⭐

**문제**: Q8의 테슬라 데이터에 다음 요소를 모두 포함하는 차트를 만드세요:

1. Candlestick 차트
2. MA7, MA21 이동평균선
3. 볼린저 밴드 (20일, 2σ)
4. 골든 크로스 / 데드 크로스 마커
5. 거래량 (하단 별도 차트, `make_subplots` 사용)

**힌트**: `make_subplots(rows=2, cols=1)` 활용

In [ ]:
# 여기에 코드를 작성하세요


### Q10. 암호화폐 비교 대시보드 ⭐⭐⭐⭐⭐

**문제**: **비트코인(BTC-USD)**과 **이더리움(ETH-USD)**의 최근 3개월 데이터를 수집하고, **2×2 서브플롯**으로 비교 대시보드를 만드세요.

**레이아웃**:
- (1,1): BTC Candlestick + MA
- (1,2): ETH Candlestick + MA
- (2,1): BTC 거래량
- (2,2): ETH 거래량

**추가 요구사항**:
- 각 차트에 적절한 제목 추가
- 최고가, 최저가, 현재가 출력

In [ ]:
# 여기에 코드를 작성하세요


---

## 📊 학습 정리

### Part 1: 기초 핵심 요약

| 기능 | 파라미터/메소드 | 실무 활용 |
|------|----------------|----------|
| Range Slider | `update_xaxes(rangeslider_visible=True)` | 대규모 시계열 데이터 탐색 |
| Range Selector | `rangeselector=dict(buttons=[...])` | 1개월/3개월/전체 빠른 전환 |
| 다중 Y축 | `yaxis='y1'`, `yaxis2=dict(overlaying='y')` | 단위가 다른 지표 동시 표시 |
| Annotation | `add_annotation()`, `add_vline()` | 중요 이벤트 강조 |
| Candlestick | `go.Candlestick(open, high, low, close)` | 금융 시장 OHLC 표현 |

### Part 2: 심화 핵심 요약

| 기법 | 계산 방법 | 해석 |
|------|----------|------|
| 이동평균(MA) | `df['MA'] = df['Close'].rolling(window=N).mean()` | 추세 파악, 매매 신호 |
| 골든 크로스 | 단기 MA > 장기 MA 교차 | 상승 추세 전환, 매수 신호 |
| 데드 크로스 | 단기 MA < 장기 MA 교차 | 하락 추세 전환, 매도 신호 |
| 볼린저 밴드 | 상단: MA + 2σ, 하단: MA - 2σ | 과매수/과매도 판단 |
| yfinance | `yf.Ticker("AAPL").history(period="3mo")` | 실시간 금융 데이터 수집 |

### 💡 실무 팁

1. **Range Slider 활용**: 1년 이상 데이터는 필수! 사용자 탐색 편의성 향상
2. **다중 Y축 주의**: 축 스케일 차이가 클 때만 사용, 혼란 방지
3. **이동평균 조합**: 단기(5/7일) + 중기(20/21일) + 장기(60/120일) 세트로
4. **볼린저 밴드 확장**: 폭이 좁아지면 큰 변동성 직전 신호
5. **백테스팅 필수**: 매매 신호는 과거 데이터로 검증 후 실전 적용
6. **API 제한 고려**: yfinance는 무료지만 과도한 요청 시 제한 가능
7. **데이터 검증**: 수집 후 결측치(NaN) 확인, `dropna()` 또는 `fillna()` 처리
8. **차트 성능**: 10,000개 이상 데이터는 `scattergl` 사용 권장